## 1. Import Dependencies

In [17]:
import keras
import os
from nltk.stem import WordNetLemmatizer
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
# from pycricbuzz import Cricbuzz
# import billboard
import time

## 2. Constants and Paths

In [20]:
DATA_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

'/home/jarisfenner/code/Kaaykun/KayBot/data'

In [12]:
lemmatizer=WordNetLemmatizer()


In [13]:
words=[]
classes=[]
documents=[]
ignore=['?','!',',',"'s"]

In [22]:
data_file=open(os.path.join(DATA_PATH, 'intents.json')).read()
intents=json.loads(data_file)